In [86]:
from datetime import date
from pathlib import Path

from loguru import logger
import sentier_data_tools as sdt

import os 

import brightway2 as bw
import lca_algebraic as agb
import numpy as np
from sympy import init_printing
import bw2io
import bw2data as bd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', 500)
from sentier_data_tools import (
    Demand,
    Flow,
    FlowIRI,
    GeonamesIRI,
    ModelTermIRI,
    DatasetKind,
    ProductIRI,
    SentierModel,
)
# Pretty print for Sympy
init_printing()

In [2]:
ECOINVENT_LOGIN="YOUR_LOGIN" #suppress login before any git push
ECOINVENT_PASSWORD="YOUR_PASSWORD"   #suppress password before any git push

In [3]:
bw.projects.set_current('PCB_Project')


# This load .env file into os.environ
load_dotenv()

# This downloads ecoinvent and installs biopshere + technosphere + LCIA methods
if len(bw.databases) > 0:
    print("Initial setup already done, skipping")
   
else:
    # This is now the prefered method to init an Brightway2 with Ecoinvent
    # It is not more tied to a specific version of bw2io
    bw2io.import_ecoinvent_release(
        version="3.10",
        system_model="cutoff",
        username=ECOINVENT_LOGIN, # Read for .env file
        password=ECOINVENT_PASSWORD, # Read from .env file
        use_mp=True)
bw.databases

Initial setup already done, skipping


Databases dictionary with 3 object(s):
	MyForeground
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [57]:
EcoInvent = "ecoinvent-3.10-cutoff"
USER_DB = 'MyForeground'
#agb.setForeground(DB_Buck)
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)
# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

[WARNING] Db MyForeground was here. Reseting it


,backend,nb_activities,type
name,,,
ecoinvent-3.10-biosphere,sqlite,4363,biosphere
ecoinvent-3.10-cutoff,sqlite,23523,background
MyForeground,sqlite,0,foreground


In [102]:
class PCB_Model(SentierModel):
    provides = {
        ProductIRI(
            "link"
        ): "Printed Wiring Board",
    }
    needs = {
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/layers_number"
        ): "layers_number",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/base_material"
        ): "base_material",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/percent_fiber_glass"
        ): "percent_fiber_glass",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/board_thickness"
        ): "board_thickness",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/copper_thickness"
        ): "copper_thickness",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/mounting_type"
        ): "mounting_type",
        
        ProductIRI("https://vocab.sentier.dev/products/pcb"): "Electronic_factory",
    }

    def get_electrolysis_inventory(self) -> None:
        bom_electrolysis = self.get_model_data(self, product=self.hydrogen, kind=DatasetKind.BOM)

    def run(self) -> tuple[list[Demand], list[Flow]]:
        self.prepare()

In [ ]:
def create_pcb_local_datastorage(reset: bool = True):
    if reset:
        sdt.reset_local_database()

    df = pd.read_excel('lci-MyForeground.xlsx')
    assert len(COLUMNS) == len(UNITS)
    assert len(COLUMNS) == len(df.columns) - 1

    metadata = sdt.Datapackage(
        name="pcb production data from EcoInvent3.10",
        description="inventory taken in EcoInvent3.10 database",
        contributors=[
            {
                "title": "Yannis Rosset",
                "path": "I don't have any website !",
                "role": "author",
            },
        ],
        homepage="https://ecoquery.ecoinvent.org/3.10/cutoff/dataset/9764/documentation",
    ).metadata()
    metadata.pop("version")

#    for kind, iri in TYPES:
#        filtered = df[df["Electrolysis type"] == kind].copy()
#        logger.info("Adding {} records for type {}", len(filtered), kind)
 #       filtered.drop(labels=["Electrolysis type"], axis="columns", inplace=True)
#        filtered.columns = COLUMNS

    sdt.Dataset(
        name=f"pcb process model",
        dataframe=df,
        product=iri,
        columns=[{"iri": x, "unit": y} for x, y in zip(COLUMNS, UNITS)],
        metadata=metadata,
        location="https://www.geonames.org/6295630/",
        version=1,
        valid_from=date(2018, 1, 1),
        valid_to=date(2028, 1, 1),
    ).save()

    for key, value in LIFETIMES.items():
        sdt.Dataset(
            name="Estimated electrolyzer BoP (balance of plant) lifetimes",
            dataframe=pd.DataFrame([{key: value}]),
            product=key,
            columns=[
                {
                    "iri": "https://vocab.sentier.dev/model-terms/electrolyser/product_lifetime",
                    "unit": "https://vocab.sentier.dev/units/unit/YR",
                }
            ],
            location="https://sws.geonames.org/6255148/",
            metadata=metadata,
            version=1,
            valid_from=date(2018, 1, 1),
            valid_to=date(2028, 1, 1),
        ).save()

    sdt.Dataset(
        name="Estimated PEM electrolyzer Stack materials",
        dataframe=pd.DataFrame(PEM_STACK),
        product="https://vocab.sentier.dev/products/pem-electrolyzer",
        columns=PEM_STACK_COLUMNS,
        location="https://sws.geonames.org/6255148/",
        metadata=metadata
        | {
            "determining_value": "https://vocab.sentier.dev/model-terms/energy/nom_power_cons"
        },
        version=1,
        kind=sdt.DatasetKind.BOM,
        valid_from=date(2018, 1, 1),
        valid_to=date(2028, 1, 1),
    ).save()

    sdt.Dataset(
        name="electrolysis water consumption value",
        dataframe=pd.DataFrame(
            [
                {
                    "http://data.europa.eu/xsp/cn2024/285390100080": 14.0,
                    "http://openenergy-platform.org/ontology/oeo/OEO_00010379": 1.0,
                },
            ]
        ),
        product="http://openenergy-platform.org/ontology/oeo/OEO_00010379",
        columns=[
            {
                "iri": "http://data.europa.eu/xsp/cn2024/285390100080",
                "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
            },
            {
                "iri": "http://openenergy-platform.org/ontology/oeo/OEO_00010379",
                "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
            },
        ],
        metadata={
            "description": "...water needs per kg of H2 are reported.. ranging from 10.01 to 22.40 l per kg of H2.",
            "homepage": "https://www.sciencedirect.com/science/article/abs/pii/S0959652621023428",
            "contributors": [
                {
                    "title": "Sofia Simoes",
                    "path": "https://orcid.org/0000-0003-4304-1411",
                    "role": "author",
                },
                {
                    "title": "Chris Mutel",
                    "path": "https://chris.mutel.org/",
                    "role": "wrangler",
                },
            ],
            "determining_value": "http://openenergy-platform.org/ontology/oeo/OEO_00010379",
        },
        version=1,
        kind=sdt.DatasetKind.BOM,
        valid_from=date(2018, 1, 1),
        valid_to=date(2028, 1, 1),
    ).save()


COLUMNS = [
    "https://vocab.sentier.dev/model-terms/generic/company",
    "https://vocab.sentier.dev/model-terms/generic/product",
    "https://vocab.sentier.dev/model-terms/energy/min_power_cons",
    "https://vocab.sentier.dev/model-terms/energy/nom_power_cons",
    "https://vocab.sentier.dev/model-terms/energy/max_power_cons",
    "https://vocab.sentier.dev/model-terms/energy/input_voltage",
    "https://vocab.sentier.dev/model-terms/generic/footprint_area",
    "https://vocab.sentier.dev/model-terms/generic/availability",
    "https://vocab.sentier.dev/model-terms/electrolyzer/min_amb_temp",
    "https://vocab.sentier.dev/model-terms/electrolyzer/max_amb_temp",
    "https://vocab.sentier.dev/model-terms/energy/elec_energy_serv_dem",
    "https://vocab.sentier.dev/model-terms/energy/therm_energy_serv_dem",
    "https://vocab.sentier.dev/model-terms/energy/therm_energy_conv_eff",
    "https://vocab.sentier.dev/model-terms/electrolyzer/temp_useful_heat",
    "https://vocab.sentier.dev/model-terms/energy/energy_conv_eff_lhv",
    "http://openenergy-platform.org/ontology/oeo/OEO_00140049",
    "https://vocab.sentier.dev/model-terms/electrolyzer/min_stack_temp",
    "https://vocab.sentier.dev/model-terms/electrolyzer/max_stack_temp",
    "https://vocab.sentier.dev/model-terms/electrolyzer/max_water_conduc",
    "https://vocab.sentier.dev/model-terms/electrolyser/max_stack_lifetime",
    "https://vocab.sentier.dev/model-terms/electrolyser/h2_quality",
    "https://vocab.sentier.dev/model-terms/electrolyser/h2_pressure",
    "https://vocab.sentier.dev/model-terms/generic/mass_prod_rate",
]

UNITS = [
    "https://www.w3.org/2001/XMLSchema#string",
    "https://www.w3.org/2001/XMLSchema#string",
    "https://vocab.sentier.dev/units/unit/KiloW",
    "https://vocab.sentier.dev/units/unit/KiloW",
    "https://vocab.sentier.dev/units/unit/KiloW",
    "https://vocab.sentier.dev/units/unit/V",
    "https://vocab.sentier.dev/units/unit/M2",
    "https://vocab.sentier.dev/units/unit/FRACTION",
    "https://vocab.sentier.dev/units/unit/DEG_C",
    "https://vocab.sentier.dev/units/unit/DEG_C",
    "https://vocab.sentier.dev/units/unit/KiloW-HR-PER-KiloGM",
    "https://vocab.sentier.dev/units/unit/MegaJ-PER-KiloGM",
    "https://vocab.sentier.dev/units/unit/FRACTION",
    "https://vocab.sentier.dev/units/unit/DEG_C",
    "https://vocab.sentier.dev/units/unit/PERCENT",
    "https://vocab.sentier.dev/units/unit/PERCENT",
    "https://vocab.sentier.dev/units/unit/DEG_C",
    "https://vocab.sentier.dev/units/unit/DEG_C",
    "https://vocab.sentier.dev/units/unit/MicroS-PER-CentiM",
    "https://vocab.sentier.dev/units/unit/HR",
    "https://vocab.sentier.dev/units/unit/NUM",
    "https://vocab.sentier.dev/units/unit/PA",
    "https://vocab.sentier.dev/units/unit/KiloGM-PER-HR",
]

TYPES = [
    ("PEM", "https://vocab.sentier.dev/products/pem-electrolyzer"),
    ("AEC", "https://vocab.sentier.dev/products/aec-electrolyzer"),
    ("SOEC", "https://vocab.sentier.dev/products/soel-electrolyzer"),
]

LIFETIMES = {
    "https://vocab.sentier.dev/products/pem-electrolyzer": 20,
    "https://vocab.sentier.dev/products/aec-electrolyzer": 27.5,
    "https://vocab.sentier.dev/products/soel-electrolyzer": 20,
}

PEM_STACK = [
    {
        "https://vocab.sentier.dev/model-terms/electrolyser/stack": 1.00,
        "http://data.europa.eu/xsp/cn2024/760611000010": 27.00,
        "http://data.europa.eu/xsp/cn2024/810890500080": 528.00,
        "https://vocab.sentier.dev/products/tetrafluoroethylene": 16.00,
        "http://data.europa.eu/xsp/cn2024/280300000080": 4.50,
        "http://data.europa.eu/xsp/cn2024/711041000010": 0.80,
        "http://data.europa.eu/xsp/cn2024/711019100080": 0.08,
        "http://data.europa.eu/xsp/cn2024/280300000080": 4.50,
        "http://data.europa.eu/xsp/cn2024/740900000080": 4.50,
        "http://data.europa.eu/xsp/cn2024/722000000080": 100.00,
        "http://data.europa.eu/xsp/cn2024/401693000080": 4.80,
        "http://data.europa.eu/xsp/cn2024/271600000080": 103890.77,
    }
]
PEM_STACK_COLUMNS = [
    {
        "iri": "https://vocab.sentier.dev/model-terms/energy/nom_power_cons",
        "unit": "https://vocab.sentier.dev/units/unit/MegaW",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/760611000010",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "end plate",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/810890500080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "bipolar plate",
    },
    {
        "iri": "https://vocab.sentier.dev/products/tetrafluoroethylene",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "comment": "membrane polymer",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/280300000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "electrocatalyst anode",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/711041000010",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "electrocatalyst anode",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/711019100080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "electrocatalyst cathode",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/280300000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "electrocatalyst cathode",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/740900000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "assembly": "current collector",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/722000000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "comment": "bolts and screws",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/401693000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloGM",
        "comment": "gasket",
    },
    {
        "iri": "http://data.europa.eu/xsp/cn2024/271600000080",
        "unit": "https://vocab.sentier.dev/units/unit/KiloW-HR",
        "comment": "assembly energy",
    },
]

In [100]:
from pydantic import BaseModel, Field
from typing import Optional


class PCB_param(BaseModel):
    layer : Optional[int] = Field(default=4, description ="number of layers")
    base : Optional[str] = Field(default="Fiber glass / Epoxy", description ="base materials for the PCB")
    RatioBase : Optional[float] = Field(default=0.6, description ="ratio between the two main main base materials")
    BoardThickness : Optional[float] = Field(default=1.6, description ="board thickness in mm")
    CopperThickness : Optional[float] = Field(default=30, description ="copper thickness in µm")
    Finish : Optional[str] = Field(default="Ni/Au", description ="surface finish type")
    mounting : Optional[str] = Field(default="SMD", description ="mounting type")
    def create_pcb(self):
        print("coucou {self.base}")
    

In [89]:
essai = pd.read_excel('lci-MyForeground.xlsx')
essai

,name,amount,location,unit,categories,type,formula,uncertainty type,loc,scale,activity,classifications,comment,flow,pedigree,production volume,properties,scale without pedigree,URL
0,"AOX, Adsorbable Organic Halides",3.62432e-05,NaN,kilogram,water::surface water,biosphere,NaN,2,-10.2253,0.3,NaN,NaN,"Estimation, based on literature.",e395a1a3-6915-4b72-a533-ff1a82fc3da1,NaN,0,NaN,0.223607,0
1,Arsenic ion,5.91178e-07,NaN,kilogram,water::surface water,biosphere,NaN,2,-14.3411,0.830662,NaN,NaN,"Estimation, based on literature.",8c8ffaa5-84ed-4668-ba7d-80fd0f47013f,NaN,0,NaN,0.806226,http://data.europa.eu/xsp/cn2024/280480000080
2,"BOD5, Biological Oxygen Demand",0.0221811,NaN,kilogram,water::surface water,biosphere,NaN,2,-3.80852,0.3,NaN,NaN,"Estimation, based on literature.",70d467b6-115e-43c5-add2-441de9411348,NaN,0,NaN,0.223607,0
3,"COD, Chemical Oxygen Demand",0.0221811,NaN,kilogram,water::surface water,biosphere,NaN,2,-3.80852,0.3,NaN,NaN,"Estimation, based on literature.",fc0b5c85-3b49-42c2-a3fd-db7e57b696e3,NaN,0,NaN,0.223607,0
4,Cadmium II,5.00054e-06,NaN,kilogram,water::surface water,biosphere,Cd+2,2,-12.206,0.830662,NaN,NaN,"Estimation, based on literature.",af83b42f-a4e6-4457-be74-46a87798f82a,NaN,0,NaN,0.806226,http://data.europa.eu/xsp/cn2024/811269100080
5,Chromium III,1.41602e-05,NaN,kilogram,water::surface water,biosphere,Cr+3,2,-11.1651,0.830662,NaN,NaN,"Estimation, based on literature.",e34d3da4-a3d5-41be-84b5-458afe32c990,NaN,0,NaN,0.806226,http://data.europa.eu/xsp/cn2024/811221900080
6,Copper ion,0.00498,NaN,kilogram,air::urban air close to ground,biosphere,NaN,2,-5.30233,0.591608,NaN,NaN,"Calculated Value, based literature and on an a...",88cde01c-df69-40bb-9b14-6eac71bea5b8,NaN,0,NaN,0.556776,0
7,Copper ion,4.76303e-05,NaN,kilogram,water::surface water,biosphere,NaN,2,-9.95204,0.830662,NaN,NaN,"Estimation, based on literature.",6d9550e2-e670-44c1-bad8-c0c4975ffca7,NaN,0,NaN,0.806226,0
8,"DOC, Dissolved Organic Carbon",0.0082152,NaN,kilogram,water::surface water,biosphere,NaN,2,-4.80177,0.3,NaN,NaN,"Calculated Value, as equal to TOC.",960c0f37-f34c-4fc1-b77c-22d8b35fd8d5,NaN,0,NaN,0.223607,0
9,Fluoride,0.000101992,NaN,kilogram,water::surface water,biosphere,F-,2,-9.19061,0.3,NaN,NaN,"Estimation, based on literature.",00d2fef1-e4d4-4a16-8e81-b8cc514e4c25,NaN,0,NaN,0.223607,0
